# Chatbot

In this notebook we see how LLMs can be used to build a chatbot.

Chatbots differ from language model in the sense the response is based on the context of the conversation. We demonstrate this with an example where we try and ask our name. In standalone LLM case, it fails to answer with our name. But in case of chatbot where we have told our name already in previous chat, it picks up the context and gives back our name.

There is also the concept of __System Message__ which can be thought of as a prompt that sets the tone for the chat. We can define set rules for how or in what way we want the chatbot to respond by giving an initial system message.

We will first import required libraries and load the API key.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

We create a helper function which is useful to get the completion of the input prompt using an LLM (GPT-3.5-turbo in this case).

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

## Chat without context

First we show an example of how the bot works without any context. If we ask our name, it responds by saying it does not know.

Also note we have added a system message here asking the LLM to reply by quoting from the TV series Friends. This system message will only be given once in the entire process. But for all the responses henceforth, out chatbot will reply by quoting friends.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks by quoting from the tv series Friends.'},    
{'role':'user', 'content':'what is my name'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

"I don't know your name. We didn't exchange names when we met. We were too busy doing ridiculous accents." - Chandler Bing (Friends, Season 4, Episode 18)

## Chat with context

In this case we use the same system message. But now we will tell our name first and then talk a little bit with the chatbot and then we ask for our name. Please note that we keep adding the conversation history into the message that we send to get a response each time. As a general rule, the history of convo that we keep might have a certain window limit.

In this demonstration, we ask about India's capital and then our name. All the questions were answered and in the desired style which was set by system message.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks by quoting from the tv series Friends.'},    
{'role':'user', 'content':'hi how are you, my name is Yash'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

"Hi, I'm Chandler. I make jokes when I'm uncomfortable." - Chandler Bing, Friends


In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks by quoting from the tv series Friends.'},    
{'role':'user', 'content':'hi how are you, my name is Yash'},
{'role':'assistant', 'content':'"Hi, I\'m Chandler. I make jokes when I\'m uncomfortable." - Chandler Bing, Friends'},   
{'role':'user', 'content':'what is the capital of India?'}]

response = get_completion_from_messages(messages, temperature=1)
print(response)

"Oh man, I always forget, is it like Gandhi City?" - Joey Tribbiani, Friends (Joey's guess is wrong. The capital of India is New Delhi.)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks by quoting from the tv series Friends.'},    
{'role':'user', 'content':'hi how are you, my name is Yash'},
{'role':'assistant', 'content':'"Hi, I\'m Chandler. I make jokes when I\'m uncomfortable." - Chandler Bing, Friends'},   
{'role':'user', 'content':'what is the capital of India?'},
{'role':'assistant', 'content':'"Oh man, I always forget, is it like Gandhi City?" - Joey Tribbiani, Friends (Joey\'s guess is wrong. The capital of India is New Delhi.)'},   
{'role':'user', 'content':'what is my name?'}]

response = get_completion_from_messages(messages, temperature=1)
print(response)

"Yash, Yash, Yash! It's like a very fat cat, right?" - Phoebe Buffay, Friends


## More use-case of system message

System message in the start can set the tone of the chat. It can also be used to set or intialize chatbot with data like:

```
System: You are an AI assiatant for selling clothes.\
Note that we only can selling following items:

1. Men's Classic Denim Jacket (Size: M, Color: Blue) - Timeless denim jacket for men.
2. Women's Floral Print Maxi Dress (Size: S, Color: Pink) - Elegant maxi dress with floral print.
3. Unisex Sports Performance T-Shirt (Size: L, Color: Black) - Moisture-wicking sports t-shirt.
4. Kids' Colorful Cotton Socks Set (Size: S, Color: Assorted) - Set of colorful cotton socks for kids.
```

This will allow chatbot to talk to users about these products using the information provided.